In [ ]:
# Download and install
curl https://clickhouse.com/ | sh

# Verify
clickhouse local --query "SELECT version()"

# Interactive mode
clickhouse local

In [ ]:
--via docker

services:
  clickhouse:
    image: clickhouse/clickhouse-server:latest
    ports:
      - "8123:8123"
      - "9000:9000"
    volumes:
      - clickhouse_data:/var/lib/clickhouse

volumes:
  clickhouse_data: